In [ ]:
import os
import sys
import time
import re
import jwt
import requests
import aiohttp
import asyncio
import certifi
import ssl
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
from colorama import Fore, init
# Initialize colorama for auto color reset
init(autoreset=True)
load_dotenv()

In [ ]:
train_booking_info = {
    "mobile_number": os.getenv("num"),
    "password": os.getenv("passcode"),
    "from_station": os.getenv("from_station"),
    "to_station": os.getenv("to_station"),
    "journey_date": os.getenv("journey_date"),
    "seat_class": os.getenv("seat_class"),
    "train_number": int(os.getenv("train_number")),
    "seat": int(os.getenv("seat")),
    "desired_seats": os.getenv("desired_seats", "").split(',') if os.getenv("desired_seats") else []
}


In [ ]:
def auth_token(num, passcode, max_retries=50):
    url = "https://railspaapi.shohoz.com/v1.0/app/auth/sign-in"
    payload = {
        "mobile_number": num,
        "password": passcode  
    }

    retries = 0  # Retry counter
    
    while retries < max_retries:
        try:
            res = requests.post(url, json=payload)  

            if res.status_code == 200:
                data = res.json()
                token = data.get('data', {}).get('token')
                
                if token:
                    print(Fore.GREEN + "Success:")
                    print(Fore.GREEN + "Auth Token:", token)
                    return token
                else:
                    print(Fore.RED + "Failed to retrieve auth token from response.")
                    return None
            
            elif res.status_code in [500, 502, 503, 504]:
                print(Fore.YELLOW + f"Server error {res.status_code}. Retrying in 1 second... ({retries + 1}/{max_retries})")
                time.sleep(1)
                retries += 1  # Increment retry count
            else:
                print(Fore.RED + f"Error: {res.status_code} - {res.text}")
                return None  # Exit if it's a client-side error (400, 401, etc.)

        except requests.RequestException as e:
            print(Fore.RED + f"Request error: {e}. Retrying in 1 second... ({retries + 1}/{max_retries})")
            time.sleep(1)
            retries += 1  # Increment retry count

    print(Fore.RED + "Max retries reached. Failed to obtain auth token.")
    return None  # Return None if all retries fail



In [ ]:
def extract_user_info(auth_key):
    try:
        decoded_token = jwt.decode(auth_key,options={"verify_signature":False},algorithms=['RS256'])
    
        email = decoded_token.get("email","")
        phone = decoded_token.get("phone_number","")
        name = decoded_token.get("display_name","")

        print(f"{Fore.CYAN}Email: {email}, Phone :{phone}, Name: {name}")

        return email,phone,name
    except Exception as e:
        print(f"{Fore.RED}Failed to decode auth token: {e}")
        return None,None,None


In [ ]:

def fetch_trip_details(from_city, to_city, date, seat_class, train_number):
    url = "https://railspaapi.shohoz.com/v1.0/app/bookings/search-trips-v2"
    payload = {
        "from_city": from_city,
        "to_city": to_city,
        "date_of_journey": date,
        "seat_class": seat_class
    }


    print(f"{Fore.YELLOW}Fetching trip details for {from_city} to {to_city} on {date}...")

    while True:  # Limit retries to prevent infinite loops
        try:
            response = requests.get(url, headers=headers, params=payload)
            if response.status_code == 200:
                data = response.json().get('data', {}).get('trains', [])

                if not data:
                    print(f"{Fore.YELLOW}Trip details not available yet. Retrying in 1 second...")
                    time.sleep(1)
                    continue
                for train in data:
                    if train.get('train_model') == str(train_number):
                        for seat in train.get('seat_types', []):
                            if seat.get("type") == seat_class:
                                trip_id = seat.get('trip_id')
                                route_id = seat.get('trip_route_id')
                                boarding_id = train.get('boarding_points', [{}])[0].get("trip_point_id", None)
                                train_name = train.get("trip_number")

                                print(f"{Fore.YELLOW}Trip details found! Train: {train_name}, Trip ID: {trip_id}, Route ID: {route_id}, Boarding Point ID: {boarding_id}")
                                return trip_id, route_id, boarding_id, train_name
                print(f"{Fore.YELLOW}Train Number {train_number} with seat class {seat_class} not available. Retrying in 1 second...")
            elif response.status_code in [500, 502, 503, 504]:
                print(Fore.YELLOW + f"Server error {response.status_code}. Retrying in 1 second...")
                time.sleep(1)
                retries += 1  # Increment retry count
            else:
                print(Fore.RED + f"Failed to fetch trip details: {response.status_code} - {response.text}")
                print(f"{Fore.YELLOW}Server response: {response.text}")
                time.sleep(1)

        except requests.RequestException as e:
            print(Fore.RED + f"Error during fetch trip details: {e}. Retrying in 1 second...")
            time.sleep(1)

# Example usage



In [ ]:

async def is_booking_available():
    """Continuously checks if train booking is available."""
    
    url = "https://railspaapi.shohoz.com/v1.0/app/bookings/seat-layout"
    payload = {
        "trip_id": trip_id,
        "trip_route_id": route_id
    }

    MIN_LOOP_INTERVAL = 0.001
      # Create an SSL context
    ssl_context = ssl.create_default_context(cafile=certifi.where())

    # Create a connector with the SSL context
    connector = aiohttp.TCPConnector(ssl_context=ssl_context)

    start_time = time.perf_counter()  # Initialize start time

    async with aiohttp.ClientSession(connector=connector) as session:
        while True:
            try:
                async with session.get(url, headers=headers, json=payload) as response:
                    end_time = time.perf_counter()
                    elapsed = end_time - start_time

                    if response.status == 200:
                        data = await response.json()
                       

                        # Check if "seatLayout" exists in the response
                        if "seatLayout" in data.get("data", {}):
                            print(f"{Fore.GREEN}Booking is now available!")
                            return data["data"]["seatLayout"]  # Return seat layout

                    elif response.status in [500, 501, 503, 504]:
                        print(f"{Fore.YELLOW}Server overloaded (HTTP {response.status}). Retrying...")

                    elif response.status == 422:
                        error_data = await response.json()
                        error_messages = error_data.get("error", {}).get("message", "")

                        if isinstance(error_messages, list):
                            error_message = error_messages[0]
                        elif isinstance(error_messages, dict):
                            error_message = error_messages.get("message", "")
                            error_key = error_messages.get("errorKey", "")
                        else:
                            error_message = "Unknown error"

                        print(f"{Fore.CYAN}Server response: {error_data}")

                        if "ticket purchase for this trip will be available" in error_message:
                            print(f"{Fore.YELLOW}Booking is not open yet: {error_message}. Retrying...")
                            await asyncio.sleep(MIN_LOOP_INTERVAL)
                            continue

                        if error_key == "OrderLimitExceeded":
                            print(f"{Fore.RED}Error: You have reached the maximum ticket booking limit for {train_booking_info['from_station']} to {train_booking_info['to_station']} on {train_booking_info['journey_date']}.")
                        else:
                            time_match = re.search(r"(\d+)\s*minute[s]?\s*(\d+)\s*second[s]?", error_message, re.IGNORECASE)
                            if time_match:
                                minutes = int(time_match.group(1))
                                seconds = int(time_match.group(2))
                                total_seconds = minutes * 60 + seconds

                                current_time_formatted = time.strftime("%I:%M:%S %p", time.localtime())
                                future_time_formatted = time.strftime("%I:%M:%S %p", time.localtime(time.time() + total_seconds))

                                print(f"{Fore.RED}Error: {error_message} Current system time is {current_time_formatted}. Try again after {future_time_formatted}.")
                            else:
                                print(f"{Fore.YELLOW}{error_message} Please try again later.")
                        exit()

                    else:
                        print(f"{Fore.RED}Failed to fetch seat layout. HTTP: {response.status}")
                        text_resp = await response.text()
                        print(f"{Fore.CYAN}Server response: {text_resp}")

            except aiohttp.ClientError as e:
                end_time = time.perf_counter()
                elapsed = end_time - start_time
                print(f"{Fore.RED}An error occurred while checking booking availability: {e}")

            if elapsed < MIN_LOOP_INTERVAL:
                await asyncio.sleep(MIN_LOOP_INTERVAL - elapsed)


In [ ]:
def get_ticket_id(seat_layout,desired_seats,max_seat):
    selected_seats = {}

    if desired_seats:
        for coach in seat_layout:
            for row in coach['layout']:
                for seat in row:
                   
                    if(seat['seat_availability'] == 1 and seat['seat_number'] in desired_seats):
                        selected_seats[seat['ticket_id']] = seat['seat_number']
                        if(len(selected_seats)==max_seat):
                            return selected_seats
                for coach in seat_layout:
                    for row in coach['layout']:
                        seat_numbers = [seat for seat in row if seat['seat_availability'] == 1]

                        for desired_seat in desired_seats:
                            nearby_seats = [s for s in seat_numbers if s['seat_number'] == desired_seat]
                        
                            if nearby_seats:
                                desired_index = seat_numbers.index(nearby_seats[0])

                                for offset in range(1,len(seat_numbers)):
                                    if desired_index + offset < len(seat_numbers):
                                        seat = seat_numbers[desired_index + offset]
                                        if(seat['seat_availability']==1 and seat['seat_number'] not in selected_seats.values()):
                                            selected_seats[seat['ticket_id']] = seat['seat_number']
                                            if(len(selected_seats) == max_seat):
                                                return selected_seats
                                    
                                    if desired_index - offset >=0:
                                        seat = seat_numbers[desired_index - offset]
                                        if seat['seat_availability'] == 1 and seat['seat_number'] not in selected_seats.values():
                                            selected_seats[seat['ticket_id']] = seat['seat_number']
                                            if(len(selected_seats)==max_seat):
                                                return selected_seats
                                            

                for coach in seat_layout:
                    for row in coach['layout']:
                        for seat in row:
                            if(seat['seat_availability'] == 1 and seat['seat_number'] not in selected_seats.values()):
                                selected_seats[seat['ticket_id']] = seat['seat_number']
                                if(len(selected_seats) == max_seat):
                                    return selected_seats


    else:
        selected_seats_list = []
        all_available_seats = []

        for layout in seat_layout:
            coach_name = layout['floor_name']
            seats = [seat for row in layout['layout'] for seat in row if seat['seat_availability']==1]

            if seats:
                seats.sort(key = lambda x: int(x['seat_number'].split('-')[-1]))
                all_available_seats.append({'coach': coach_name,'seats':seats})
        
        for coach_data in all_available_seats:
            coach_name = coach_data['coach']
            seats = coach_data['seats']


            mid_index = len(seats) //2
            for i in range(max(0,mid_index-max_seat),min(mid_index+1,len(seats) - max_seat+1)):
                block = seats[i:i + max_seat]
                seat_numbers = [int(seat['seat_number'].split('-')[-1]) for seat in block]


                if max(seat_numbers) - min(seat_numbers) == max_seat - 1:
                    for seat in block:
                        selected_seats[seat['ticket_id']] = seat['seat_number']
            

        for coach_data in all_available_seats:
            if(len(selected_seats_list) >= max_seat):
                break

            coach_name = coach_data['coach']
            ets = coach_data['seats']
            coach_selected_seats = []

            mid_index = len(seats)//2
            left = mid_index-1
            right = mid_index

            while(len(selected_seats_list) < max_seat and (left>=0 or right <len(seats))):
                if left >=0 and len(selected_seats_list) < max_seat:
                    coach_selected_seats.append(seats[left])
                    selected_seats_list.append(seats[left])
                    left -= 1

                if right<len(seats) and len(selected_seats_list) < max_seat:
                    coach_selected_seats.append(seats[right])
                    selected_seats_list.append(seats[right])
                    right + 1
            if coach_selected_seats:
                for seat in coach_selected_seats:
                    selected_seats[seat['ticket_id']] = seat['seat_number']
                if(len(selected_seats) >= max_seat):
                    return selected_seats
        

        if(len(selected_seats) < max_seat):
            for coach_data in all_available_seats:
                if(len(selected_seats_list) >= max_seat):
                    break

                coach_name = coach_data['coach']
                seats = coach_data['seats']

                for seat in seats:
                    if len(selected_seats_list) > max_seat:
                        break
                    if seat not in selected_seats.values():
                        selected_seats[seat['ticket_id']] = seat['seat_number']
                        selected_seats_list.append(seat)


    if selected_seats:
        print(f"{Fore.YELLOW}Warning: Proceeding with {len(selected_seats)} instead of {max_seat}.")
        return selected_seats
    
    print(f"{Fore.RED} No seats available to proceed.")
    return None






In [ ]:

# def find_selected_seats(seat_layout, desired_seats, max_seat):
#     """
#     Finds the selected seats based on the desired seat numbers.
#     This assumes that `desired_seats` is a list of desired seat numbers.
#     """
#     selected_seats = {}

#     # Look for seats matching desired seat numbers
#     for coach in seat_layout:
#         for row in coach['layout']:
#             for seat in row:
#                 if seat['seat_availability'] == 1 and seat['seat_number'] in desired_seats:
#                     selected_seats[seat['ticket_id']] = seat['seat_number']
#                     if len(selected_seats) == max_seat:
#                         return selected_seats
#     return selected_seats


# def find_nearby_seats(seat_layout, desired_seats, max_seat):
#     """
#     Finds nearby seats based on available seats close to the desired ones.
#     """
#     selected_seats = {}

#     for coach in seat_layout:
#         for row in coach['layout']:
#             seat_numbers = [seat for seat in row if seat['seat_availability'] == 1]

#             for desired_seat in desired_seats:
#                 nearby_seats = [s for s in seat_numbers if s['seat_number'] == desired_seat]
#                 if nearby_seats:
#                     desired_index = seat_numbers.index(nearby_seats[0])

#                     # Find seats to the right of the desired seat
#                     selected_seats = select_nearby_seats(selected_seats, seat_numbers, desired_index, max_seat, 1)
#                     if len(selected_seats) == max_seat:
#                         return selected_seats

#                     # Find seats to the left of the desired seat
#                     selected_seats = select_nearby_seats(selected_seats, seat_numbers, desired_index, max_seat, -1)
#                     if len(selected_seats) == max_seat:
#                         return selected_seats

#     return selected_seats


# def select_nearby_seats(selected_seats, seat_numbers, start_index, max_seat, direction):
#     """
#     Helper function to select nearby seats based on the given direction (left or right).
#     """
#     for offset in range(1, len(seat_numbers)):
#         next_index = start_index + (offset * direction)
#         if 0 <= next_index < len(seat_numbers):
#             seat = seat_numbers[next_index]
#             if seat['seat_availability'] == 1 and seat['seat_number'] not in selected_seats.values():
#                 selected_seats[seat['ticket_id']] = seat['seat_number']
#                 if len(selected_seats) == max_seat:
#                     return selected_seats
#     return selected_seats


# def find_remaining_seats(seat_layout, selected_seats, max_seat):
#     """
#     Select remaining seats if the previously selected seats are not enough.
#     """
#     for coach in seat_layout:
#         for row in coach['layout']:
#             for seat in row:
#                 if seat['seat_availability'] == 1 and seat['seat_number'] not in selected_seats.values():
#                     selected_seats[seat['ticket_id']] = seat['seat_number']
#                     if len(selected_seats) == max_seat:
#                         return selected_seats
#     return selected_seats


# def find_seat_blocks(seat_layout, max_seat):
#     """
#     Finds contiguous blocks of available seats in the layout.
#     """
#     all_available_seats = []
#     for layout in seat_layout:
#         coach_name = layout['floor_name']
#         seats = [seat for row in layout['layout'] for seat in row if seat['seat_availability'] == 1]
        
#         if seats:
#             seats.sort(key=lambda x: int(x['seat_number'].split('-')[-1]))
#             all_available_seats.append({'coach': coach_name, 'seats': seats})

#     return all_available_seats


# def get_ticket_id(seat_layout, desired_seats, max_seat):
#     """
#     Main function to select seats from the seat layout.
#     """
#     selected_seats = {}

#     if desired_seats:
#         selected_seats = find_selected_seats(seat_layout, desired_seats, max_seat)

#         if len(selected_seats) < max_seat:
#             selected_seats = find_nearby_seats(seat_layout, desired_seats, max_seat)

#         if len(selected_seats) < max_seat:
#             selected_seats = find_remaining_seats(seat_layout, selected_seats, max_seat)

#     else:
#         all_available_seats = find_seat_blocks(seat_layout, max_seat)
#         selected_seats = select_block_seats(all_available_seats, max_seat)

#     if selected_seats:
#         print(f"{Fore.YELLOW}Warning: Proceeding with {len(selected_seats)} instead of {max_seat}.")
#         return selected_seats

#     print(f"{Fore.RED} No seats available to proceed.")
#     return None


# def select_block_seats(all_available_seats, max_seat):
#     """
#     Selects contiguous block seats when no specific seat numbers are provided.
#     """
#     selected_seats = {}
#     selected_seats_list = []

#     for coach_data in all_available_seats:
#         coach_name = coach_data['coach']
#         seats = coach_data['seats']

#         mid_index = len(seats) // 2
#         for i in range(max(0, mid_index - max_seat), min(mid_index + 1, len(seats) - max_seat + 1)):
#             block = seats[i:i + max_seat]
#             seat_numbers = [int(seat['seat_number'].split('-')[-1]) for seat in block]

#             # If seats form a contiguous block
#             if max(seat_numbers) - min(seat_numbers) == max_seat - 1:
#                 for seat in block:
#                     selected_seats[seat['ticket_id']] = seat['seat_number']

#     for coach_data in all_available_seats:
#         if len(selected_seats_list) >= max_seat:
#             break

#         coach_name = coach_data['coach']
#         seats = coach_data['seats']

#         left = mid_index - 1
#         right = mid_index

#         while len(selected_seats_list) < max_seat and (left >= 0 or right < len(seats)):
#             if left >= 0 and len(selected_seats_list) < max_seat:
#                 selected_seats_list.append(seats[left])
#                 selected_seats[seats[left]['ticket_id']] = seats[left]['seat_number']
#                 left -= 1

#             if right < len(seats) and len(selected_seats_list) < max_seat:
#                 selected_seats_list.append(seats[right])
#                 selected_seats[seats[right]['ticket_id']] = seats[right]['seat_number']
#                 right += 1

#     return selected_seats


In [ ]:
async def reserve_seat():
    global ticket_ids
    
    print(f"{Fore.YELLOW} waiting for seat layout avalabilty...")

    seat_layout = await is_booking_available()

    if not seat_layout:
        print(f"{Fore.RED} Seat layout could not be retrieved. Exiting")
        return False
    
    ticket_id_map = get_ticket_id(seat_layout,train_booking_info['desired_seats'],train_booking_info['seat'])

    if not ticket_id_map:
        print(f"{Fore.RED}No matching seats found based on desired preferences. Exiting..")
        return False
    
    ticket_ids = list(ticket_id_map.keys())
    print(f"{Fore.GREEN}Seats matched Details : {', '.join([f'{ticket_id_map[ticket]} (Ticket ID: {ticket})' for ticket in ticket_ids])}")

    successful_ticket_ids = []
    stop_reservation_due_to_limit = False

    def reserve_single_seat(ticket):
        nonlocal stop_reservation_due_to_limit
        if stop_reservation_due_to_limit:
            return False
        
        url = "https://railspaapi.shohoz.com/v1.0/app/bookings/reserve-seat"
        payload = {
            "ticket_id":ticket,
            "route_id" : route_id
        }

        while True:
            try:
                response = requests.patch(url,headers=headers,json=payload)
                print(f"{Fore.CYAN}Response from Reserve Seat API for seta {ticket_id_map[ticket]} (Ticket Id:{ticket}): {response.text}")

                if response.status_code == 200:
                    data = response.json()

                    if data['data'].get("ack") == 1:
                        print(f"{Fore.GREEN} Seat {ticket_id_map[ticket]} (ticket ID :{ticket}) reserved succesfully")
                        successful_ticket_ids.append(ticket)
                        return True
                    else:
                        print(f"{Fore.ReD}Failed to reserve seat {ticket_id_map[ticket]} (Ticket Id: {ticket}) : {data}")
                        return False

                elif response.status_code == 422:
                    error_data = response.json()
                    error_msg = error_data.get("error",{}).get("messages",{}).get("error_msg","")
                    if "Maximum 4 seats can be booked at a time" in error_msg:
                        print(f"{Fore.RED}ERror : {error_msg}. Stopping further seat reservation.")
                        stop_reservation_due_to_limit = True
                    elif "Sorry! this ticket is not available now." in error_msg:
                        print(f"{Fore.RED}Seat {ticket_id_map[ticket]} (Ticket ID :{ticket}) is not available now. Skipping retry...")
                        return False
                elif response.status_code in [500,502,503,504]:
                    print(f"{Fore.YELLOW}Server Overloaded (HTTP {response.status_code}). Retrying in 100miliseconds....")
                    time.sleep(0.1)
                else:
                    print(f"{Fore.RED}Error :{response.status_code} - {response.text}")
                    return False
            
            except Exception as e:
                print(f"{Fore.RED}Exception occured while reserving seat {ticket_id_map[ticket]} (Ticket ID: {ticket}): {e}")
                time.sleep(0.1)
            

    print(f"{Fore.YELLOW}Intiating seat reservation process for{len(ticket_ids)} tickets...")

    with ThreadPoolExecutor(max_workers=len(ticket_ids)) as executor:
        executor.map(reserve_single_seat,ticket_ids)


    if successful_ticket_ids:
        ticket_ids = successful_ticket_ids
        print(f"{Fore.GREEN}Successfully reserved tickets {ticket_ids}. Proceeding to next step...")
        return True
    else:
        print(f"{Fore.RED} No seats could be reserved. Please try again...")
        return False




In [ ]:
def send_passenger_details():
    url = "https://railspaapi.shohoz.com/v1.0/app/bookings/passenger-details"
    payload = {
        "trip_id" : trip_id,
        "trip_route_id" : route_id,
        "ticket_ids":ticket_ids
    }

    while True:
        try:
            response = requests.post(url, headers=headers,json = payload)
            print(f"{Fore.CYAN}Response from Passenger details API: {response.text}")
            if response.status_code == 200:
                data = response.json()
                if data["data"]["success"]:
                    print(f"{Fore.GREEN}OTP sent successfully!")
                    return True
                else:
                    print(f"{Fore.RED}Failed to send OTP : {data}")
                    return False
            elif response.status_code in [500,502,503,504]:
                print(f"{Fore.YELLOW}Server Overloaded (HTTP {response.status_code}). Retrying in 1 second....")
                time.sleep(1)
            else:
                print(f"{Fore.RED}Error : {response.status_code} - {response.text}")
                return False
        
        except requests.RequestException as e:
            print(f"{Fore.RED}Exception occure while sending passenger details: {e}")
            time.sleep(1)

In [ ]:
def verify_and_confirm(otp):
    # OTP Verification URL
    verify_url = "https://railspaapi.shohoz.com/v1.0/app/bookings/verify-otp"
    verify_payload = {
        "trip_id": trip_id,
        "trip_route_id": route_id,
        "ticket_ids": ticket_ids,
        "otp": otp
    }

    try:
        while True:
            response = requests.post(verify_url, headers=headers, json=verify_payload)

            if response.status_code == 200:
                data = response.json()
                if not data["data"]["success"]:
                    print(f"{Fore.RED}Failed to verify OTP: {data}")
                    return False
                print(f"{Fore.GREEN}OTP verified Successfully!")
                break  # Exit the loop if OTP is verified

            elif response.status_code in [500, 502, 503, 504]:
                print(f"{Fore.YELLOW}Server overloaded (HTTP {response.status_code}). Retrying in 1 second...")
                time.sleep(1)

            elif response.status_code == 422:
                data = response.json()
                error_info = data.get("error", {}).get("messages", {})

                error_message = error_info.get("message", "Unknown error")
                error_key = error_info.get("errorKey", "Unknown errorkey")

                print(f"{Fore.RED}Error: {error_message} (Error Key: {error_key})")

                if error_key == "OtpNotVerified":
                    otp = input(f"{Fore.YELLOW}The OTP does not match. Please enter correct OTP: ")
                    verify_payload["otp"] = otp
                else:
                    print(f"{Fore.RED}Error: {response.status_code} - {response.text}")
                    return False

    except Exception as e:
        print(f"{Fore.RED}Exception occurred: {e}")
        time.sleep(1)
        return False

    # Booking Confirmation URL
    confirm_url = "https://railspaapi.shohoz.com/v1.0/app/bookings/confirm"
    confirm_payload = prepare_confirm_payload(otp)

    # Payment Method Selection
    print(f"\n{Fore.CYAN}Select Payment Method:")
    print(f"1. bKash\n2. Nagad\n3. Rocket\n4. Upay\n5. VISA\n6. Mastercard\n7. DBBL Nexus")

    while True:
        payment_choice = input(f"{Fore.YELLOW}Enter the number corresponding to your payment method: ")

        if payment_choice == '1':  # bKash (default)
            print(f"{Fore.GREEN}Payment Method Selected: bKash")
            break
        elif payment_choice == '2':  # Nagad
            confirm_payload["is_bkash_online"] = False
            confirm_payload["selected_mobile_transaction"] = 3
            print(f"{Fore.GREEN}Payment Method Selected: Nagad")
            break
        elif payment_choice == '3':  # Rocket
            confirm_payload["is_bkash_online"] = False
            confirm_payload["selected_mobile_transaction"] = 4
            print(f"{Fore.GREEN}Payment Method Selected: Rocket")
            break
        elif payment_choice == '4':  # Upay
            confirm_payload["is_bkash_online"] = False
            confirm_payload["selected_mobile_transaction"] = 5
            print(f"{Fore.GREEN}Payment Method Selected: Upay")
            break
        elif payment_choice == '5':  # VISA
            confirm_payload["is_bkash_online"] = False
            confirm_payload.pop("selected_mobile_transaction", None)
            confirm_payload["pg"] = "visa"
            print(f"{Fore.GREEN}Payment Method Selected: VISA")
            break
        elif payment_choice == '6':  # Mastercard
            confirm_payload["is_bkash_online"] = False
            confirm_payload.pop("selected_mobile_transaction", None)
            confirm_payload["pg"] = "mastercard"
            print(f"{Fore.GREEN}Payment Method Selected: Mastercard")
            break
        elif payment_choice == '7':  # DBBL Nexus
            confirm_payload["is_bkash_online"] = False
            confirm_payload.pop("selected_mobile_transaction", None)
            confirm_payload["pg"] = "nexus"
            print(f"{Fore.GREEN}Payment Method Selected: DBBL Nexus")
            break
        else:
            print(f"{Fore.RED}Invalid selection! Please enter a number between 1 and 7.")

    # Booking Confirmation
    while True:
        response = requests.patch(confirm_url, headers=headers, json=confirm_payload)
        print(f"{Fore.CYAN}Response from Confirm Booking API: {response.text}")

        try:
            if response.status_code == 200:
                data = response.json()
                if "redirectUrl" in data["data"]:
                    redirect_url = data["data"]["redirectUrl"]
                    print(f"\n{Fore.GREEN}{'='*50}")
                    print(f"{Fore.GREEN}Booking confirmed successfully!")
                    print(f"{Fore.YELLOW}IMPORTANT: Please note that this payment link can be used ONLY ONCE.")
                    print(f"{Fore.BLUE}Payment URL: {redirect_url}")
                    print(f"{Fore.GREEN}{'='*50}\n")
                    return True  # Ensure successful return
                else:
                    print(f"{Fore.RED}Failed to confirm booking: {data}")
                    return False

            elif response.status_code in [500, 502, 503, 504]:
                print(f"{Fore.YELLOW}Server overloaded (HTTP {response.status_code}). Retrying in 1 second...")
                time.sleep(1)

            else:
                print(f"{Fore.RED}Error: {response.status_code} - {response.text}")
                return False

        except requests.RequestException as e:
            print(f"{Fore.RED}Exception occurred while confirming booking: {e}")
            time.sleep(1)
            return False

In [ ]:
def prepare_confirm_payload(otp):
    user_email,user_phone,user_name = extract_user_info(token)
    if len(ticket_ids) > 1:
        passenger_names = [user_name]
        for i in range(1,len(ticket_ids)):
            passenger_name = input(f"{Fore.YELLOW}Enter passenger {i+1} name :")
            passenger_names.append(passenger_name)

        confirm_payload = {
            "is_bkash_online": True,
            "boarding_point_id": boarding_id,
            "from_city": train_booking_info['from_station'],
            "to_city": train_booking_info['to_station'],
            "date_of_journey": train_booking_info['journey_date'],
            "seat_class": train_booking_info['seat_class'],
            "passengerType": ["Adult"]*len(ticket_ids),
            "gender": ["male"] * len(ticket_ids),
            "pname": passenger_names,
            "pmobile": user_phone,
            "pemail": user_email,
            "trip_id": trip_id,
            "trip_route_id": route_id,
            "ticket_ids": ticket_ids,
            "contactperson": 0,
            "otp": otp,
            "selected_mobile_transaction": 1
        }
    else:
        confirm_payload = {
            "is_bkash_online": True,
            "boarding_point_id": boarding_id,
            "from_city": train_booking_info['from_station'],
            "to_city": train_booking_info['to_station'],
            "date_of_journey": train_booking_info['journey_date'],
            "seat_class": train_booking_info['seat_class'],
            "passengerType": ["Adult"],
            "gender": ["male"],
            "pname": [user_name],
            "pmobile": user_phone,
            "pemail": user_email,
            "trip_id": trip_id,
            "trip_route_id": route_id,
            "ticket_ids": ticket_ids,
            "contactperson": 0,
            "otp": otp,
            "selected_mobile_transaction": 1
        }

    return confirm_payload


In [ ]:


try:
    token = auth_token(train_booking_info['mobile_number'], train_booking_info['password'])
    if not token:
        print(f"{Fore.RED}Failed to fetch auth token. Exiting...")
        sys.exit(1)

    headers = {"Authorization": f"Bearer {token}"}

    trip_id, route_id, boarding_id, train_name = fetch_trip_details(
        train_booking_info['from_station'], 
        train_booking_info['to_station'], 
        train_booking_info['journey_date'], 
        train_booking_info['seat_class'], 
        train_booking_info['train_number']
    )

    if not all([trip_id, route_id, boarding_id]):
        print(f"{Fore.RED}Error: Could not fetch trip details. Please check your inputs.")
        sys.exit(1)
    
    if await reserve_seat():
        if send_passenger_details():
            print(f"{Fore.CYAN}Proceeding to OTP verification and conirmation...")
            
            otp = input(f"{Fore.YELLOW}Enter OTP received :")
            if verify_and_confirm(otp):
                print(f"{Fore.GREEN}Booking Process completed Succussfully!")
            else:
                print(f"{Fore.RED} Failed to complete booking process.")
        else:
            print(f"{Fore.RED}Failed to send passenger details and get OTP.")

    else:
        print(f"{Fore.RED}Failed to reserve the seat.")


except KeyError as e:
    print(f"{Fore.RED}Missing key in train_booking_info: {e}")
    sys.exit(1)

except Exception as e:
    print(f"{Fore.RED}An unexpected error occurred: {e}")
    sys.exit(1)

